# Herron - Do market prices respond to new information?

This notebook uses [Elon Musk's proposed acquisition of Twitter (TWTR)](https://en.wikipedia.org/wiki/Proposed_acquisition_of_Twitter_by_Elon_Musk) to show that market prices respond to information.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
plt.rcParams['figure.dpi'] = 150
%precision 4
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
import yfinance as yf
import requests_cache
session = requests_cache.CachedSession()

## Dates and Data

In [ ]:
dates = {
    'First Purchase': pd.to_datetime('2022-01-31'),
    'Purchases Announced': pd.to_datetime('2022-04-04'),
    'Bid Announced': pd.to_datetime('2022-04-14'),
    'Bid on Hold': pd.to_datetime('2022-05-13'),
    'Bid Revived': pd.to_datetime('2022-10-04')
}

In [ ]:
twtr_all = yf.download(tickers='TWTR', session=session)

In [ ]:
price_bid = 54.20

## Implied Probability of Deal Completion

Twitter's stock price is the present value of expected cash flows.
If the deal closes, the expected cash flow is the bid price.
Otherwise, the expected cash flow is the unaffected price.
So, $$P_t = p\times P_{bid} + (1-p)\times P_{unaffected},$$ where $p$ is the probability of deal completion.
We can rearrange this formula to estimate an implied probability of deal completion as $$p = \frac{P_t - P_{unaffected}}{P_{bid} - P_{unaffected}}.$$